In [15]:
# %pip install tensorflow==2.6.2

In [1]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

# fix random seed for reproducibility
np.random.seed(42)

2022-03-28 10:20:26.312229: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-28 10:20:26.312249: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.6.2


In [2]:
# Data loading
from scipy.io import loadmat

camera_delta_labelled_mat = loadmat("/home/nicolo/Scrivania/mat/camera_delta_labelled_3.mat")
windows_labelled_mat = loadmat("/home/nicolo/Scrivania/mat/windows_labelled_3.mat")

In [10]:
# Conversion to numpy array
camera_delta_labelled = np.transpose(camera_delta_labelled_mat['camera_delta_labelled'].astype('float64'))
windows_labelled = np.transpose(windows_labelled_mat['windows_labelled'].astype('float64'),(1,2,0))


print(camera_delta_labelled.shape)
print(windows_labelled.shape)


(47219, 4)
(47219, 31, 7)


In [12]:
# Matrices reshaping --> I need to reshape in order to have the order (windows , length , features).
# I cannot use reshape since it will change the order of the windows inside the matrix randomly,
# but at the same time using only rot90 will not be enough, i also need to flip since the rot90
# will invert last and first elements (last become first and viceversa)

# print(camera_delta_labelled[-1,29])
# print(windows_labelled[:,29,-1])

# a1 = np.rot90(camera_delta_labelled,1)
# a2 = np.flip(a1,0)
# camera_delta_labelled_rot = a2

# b1=np.rot90(windows_labelled, -1, axes=(0,1))
# b2=np.flip(b1,1)
# b3=np.rot90(b2, 1, axes=(1,2))
# b4=np.flip(b3,1)
# windows_labelled_rot = b4

# print(camera_delta_labelled_rot.shape)
# print(windows_labelled_rot.shape)

# # Check: 29th and 120th elements are belonging to class 1, let's see if it still true
# print(camera_delta_labelled_rot[29,-1]) #or 120
# print(windows_labelled_rot[29,-1,:])    #or 120


In [13]:
# Create the labels matrix
# Check whether the matlab file creation has been done correctly (labels extracted from both the matrices should be the same,
# and in the same position)

labels = camera_delta_labelled[:,-1]
print('Number of elements belonging to class 1: ', np.count_nonzero(labels == 1))
print('Number of elements belonging to class 0: ', np.count_nonzero(labels == 0))

labels_check = windows_labelled[:,-1,0]
print('Number of elements belonging to class 1: ', np.count_nonzero(labels_check == 1))
print('Number of elements belonging to class 0: ', np.count_nonzero(labels_check == 0))

check = labels - labels_check
print('Number of elements belonging to class 1: ', np.count_nonzero(check == 1))
print('Number of elements belonging to class 0: ', np.count_nonzero(check == 0))
np.sum(check)

Number of elements belonging to class 1:  112
Number of elements belonging to class 0:  47107
Number of elements belonging to class 1:  112
Number of elements belonging to class 0:  47107
Number of elements belonging to class 1:  0
Number of elements belonging to class 0:  47219


0.0

In [15]:
# Matrices creation (predictions and features)
# Matrix containing the delta movements (dx,dy,dz) or the class (whether we are doing classification or regression --> TO BE DONE simultaneously in the next step)

# In this case: CLASSIFICATION
delta_cam = camera_delta_labelled[:,-1:]
print('Matrix containing camera windows labels at each timestamp: ', delta_cam.shape)

# In this case: REGRESSION
# delta_cam = camera_delta_labelled[:,:-1]
# print('Matrix containing camera delta movements at each timestamp: ', delta_cam.shape)

kinematic_windows = windows_labelled[:,:-1,:]
print('Matrix containing kinematic features at each timestamp: ', kinematic_windows.shape)

Matrix containing camera windows labels at each timestamp:  (47219, 1)
Matrix containing kinematic features at each timestamp:  (47219, 30, 7)


In [16]:
input_shape = kinematic_windows.shape[1:]
output_shape = delta_cam.shape[1:]
print('Input shape: ', input_shape)
print('Output shape: ', output_shape)

Input shape:  (30, 7)
Output shape:  (1,)


In [17]:
X_train = kinematic_windows[:35000,:,:]
y_train = delta_cam[:35000,:]
X_val = kinematic_windows[35000:,:,:]
y_val = delta_cam[35000:,:]

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(35000, 30, 7)
(35000, 1)
(12219, 30, 7)
(12219, 1)


In [24]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [25]:
import tensorflow as tf
import keras.backend as K
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
from random import randrange
import matplotlib.pyplot as plt
import math
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Concatenate
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input, BatchNormalization, \
    multiply, concatenate, Flatten, Activation, dot
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import EarlyStopping
from IPython.display import FileLink, FileLinks
import warnings
import pprint

In [27]:
def build_CONV_LSTM_model(input_shape, output_shape):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    convlstm = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True))(input_layer)
    convlstm = tfkl.Conv1D(64, 3, padding='same', activation='relu')(convlstm)
    convlstm = tfkl.MaxPool1D()(convlstm)
    convlstm = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True))(convlstm)
    convlstm = tfkl.Conv1D(128, 3, padding='same', activation='relu')(convlstm)
    convlstm = tfkl.GlobalAveragePooling1D()(convlstm)
    convlstm = tfkl.Dropout(.3)(convlstm)

    # In order to predict the next values for more than one sensor,
    # we can use a Dense layer with a number given by telescope*num_sensors,
    # followed by a Reshape layer to obtain a tensor of dimension 
    # [None, telescope, num_sensors]
    output_layer = tfkl.Dense(output_shape[-1]*1, activation='sigmoid')(convlstm)
    #output_layer = tfkl.Reshape((1,output_shape[-1]))(output_layer)
    #output_layer = tfkl.Conv1D(output_shape[-1], 1, padding='same')(output_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='cnn_lstm_model')

    # Compile the model
    model.compile(loss=tfk.losses.BinaryCrossentropy(), optimizer=tfk.optimizers.Adam(lr = 1e-3),
                  metrics=tf.keras.metrics.Accuracy())
    # Return the model
    return model


model = build_CONV_LSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True)

Model: "cnn_lstm_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 30, 7)]           0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 30, 128)           36864     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 30, 64)            24640     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 15, 64)            0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 15, 128)           66048     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 15, 128)           49280     
_________________________________________________________________
global_average_pooling1d_2 ( (None, 128)            

In [28]:
import numpy as np
import math

# labels_dict : {ind_label: count_label}
# mu : parameter to tune 

def create_class_weight(labels_dict,mu=0.15):
    total = np.sum(list(labels_dict.values()))
    keys = labels_dict.keys()
    class_weights = dict()
    
    for key in keys:
        score = math.log(mu*total/float(labels_dict[key]))
        class_weights[key] = score if score > 1.0 else 1.0
    
    return class_weights

labels_dict = {0.0: 47107,
               1.0: 112}

class_weights = create_class_weight(labels_dict)
pprint.pprint(class_weights)

{0.0: 1.0, 1.0: 4.146932776767107}


In [29]:
batch_size = 526
epochs = 100

history = model.fit(
    X_train,
    y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_data = (X_val,  y_val),
    class_weight = class_weights,
    callbacks = [
        tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='min', patience=20, 
                                    restore_best_weights=True, verbose = 1),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=3,
                                        factor=0.1, min_lr=1e-7, verbose = 1)
    ]
).history

Epoch 1/100
67/67 [==============================] - 14s 149ms/step - loss: 0.0941 - accuracy: 0.0000e+00 - val_loss: 0.0238 - val_accuracy: 0.0000e+00
Epoch 2/100
67/67 [==============================] - 9s 137ms/step - loss: 0.0596 - accuracy: 0.0000e+00 - val_loss: 0.0181 - val_accuracy: 0.0000e+00
Epoch 3/100
67/67 [==============================] - 9s 136ms/step - loss: 0.0591 - accuracy: 0.0000e+00 - val_loss: 0.0154 - val_accuracy: 0.0000e+00
Epoch 4/100
67/67 [==============================] - 9s 136ms/step - loss: 0.0605 - accuracy: 0.0000e+00 - val_loss: 0.0162 - val_accuracy: 0.0000e+00
Epoch 5/100
67/67 [==============================] - 9s 139ms/step - loss: 0.0601 - accuracy: 0.0000e+00 - val_loss: 0.0231 - val_accuracy: 0.0000e+00
Epoch 6/100
67/67 [==============================] - 9s 138ms/step - loss: 0.0601 - accuracy: 0.0000e+00 - val_loss: 0.0213 - val_accuracy: 0.0000e+00

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 7/100

In [ ]:
# Penso che i problemi siano molteplici:
# - le classi sono parecchio imbalanced, quindi fa veramente fatica ad imparare 
# - non capisco se ho scritto correttamente il modello: alla seconda epoca sembra avere un miglioramento, come se ne azzeccasse una su 112 invece che 0, o simile
# - nonstante il miglioramento in training, sul validation fa comunque pena (non son riuscito a mettere balanced accuracy, e accuracy non è corretta come metrica)
# - vorrei capire se ha senso rifare un dataset più corposo per la classe 1, diciamo un dataset in cui ripeto sempre le stesse mosse per vedere se riesce ad allenare quando
#   vede movimenti ripetitivi, perchè questo dataset che ho usato viene da esperimenti lunghi in cui i movimenti, e quindi le finestre dei dati di cinematica sono, sono
#   molto diversi fra loro

#focal loss crossentropy
#tresholding
#np.transpose
#invece che acuracy, report completo con micro,macro confusion matrix